In [1]:
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import xarray as xr
import glob
import re
import rioxarray
import dask.array
from dask.distributed import Client, progress

In [2]:
client = Client(n_workers=10, threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 15
Total threads: 30,Total memory: 240.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42915,Workers: 15
Dashboard: /proxy/8787/status,Total threads: 30
Started: Just now,Total memory: 240.00 GiB
Comm: tcp://127.0.0.1:45401,Total threads: 2
Dashboard: /proxy/45383/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:43549,


In [2]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [3]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

In [4]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = get_directories_with_number_and_ending(workingPath+"1input_data/", "global")
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4-1]
print(year)

2015global


In [5]:
import geopandas as gpd
from shapely.geometry import box

shapefile_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/Emulator/input_data/EuropeBoundary.shp"
gdf = gpd.read_file(shapefile_path)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [7]:
### 0) read era5land data
# select Europe data of ERA5Land and save it as netcdf
all1 = xr.open_mfdataset(inputData+year+"/era5land/era5-land*.nc", chunks={"time":-1,"latitude":51,"longitude":51})
# even with single netcdf file, it stuck on exporting
# all1 = xr.open_dataset(inputData+year+"/era5land/era5-land_10m_u_component_of_wind_2015-01.nc", chunks={"time":-1,"latitude":51,"longitude":51})
lon_name = 'longitude'
all1['longitude_adjusted'] = xr.where(
    all1[lon_name] > 180,
    all1[lon_name] - 360,
    all1[lon_name])
all1 = (
    all1
    .swap_dims({lon_name: 'longitude_adjusted'})
    .sel(**{'longitude_adjusted': sorted(all1.longitude_adjusted)})
    .drop(lon_name))
all1 = all1.rename({'longitude_adjusted': lon_name})
all1 = all1.sel(
    latitude=slice(bbox[3], bbox[1]),  
    longitude=slice(bbox[0], bbox[2])  
)

In [8]:
all1

<xarray.Dataset>
Dimensions:    (latitude: 469, time: 744, longitude: 1002)
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-01-31T23:00:00
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(744, 20, 27), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 15:57:25 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [9]:
all1.chunks

Frozen({'time': (744,), 'latitude': (20, 51, 51, 51, 51, 51, 51, 51, 51, 41), 'longitude': (27, 51, 51, 51, 51, 51, 30, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 27)})

In [ ]:
from some_library import ProgressBar

In [ ]:
%%time
write_job = all1.to_netcdf('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5land2015_10km2',format='NETCDF4')
# write_job = all1.to_netcdf('/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/2015global/era5land/era5-land_10m_u_component_of_wind_2015-01Eu.nc',format='NETCDF4')
with ProgressBar():
    print(f"Writing to {out_file}")
    write_job.compute()

# test is the exported netcdf complete

In [6]:
lat1 = 40
lat2 = 60
lon1 = 2
lon2 = 22

In [11]:
test = xr.open_dataset(inputData+year+"/era5land/era5land2015_10km1.nc", chunks={"latitude":51, "longitude":51}) 
test = test.sel(
    latitude=slice(lat2,lat1),  
    longitude=slice(lon1, lon2)  
)

In [12]:
test

<xarray.Dataset>
Dimensions:    (latitude: 201, time: 8760, longitude: 201)
Coordinates:
  * latitude   (latitude) float32 60.0 59.9 59.8 59.7 ... 40.3 40.2 40.1 40.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 21.7 21.8 21.9 22.0
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(8760, 37, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 23:48:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [13]:
a = np.nanmean(test['u10'], axis=(1,2))
nan_mask = np.isnan(a)

# 使用 np.sum() 来统计True的数量，即NaN值的数量
nan_count = np.sum(nan_mask)

/scratch-local/qiahan.4011667/ipykernel_2324691/1541313271.py:1: RuntimeWarning: Mean of empty slice
  a = np.nanmean(test['u10'], axis=(1,2))


In [14]:
nan_count

7344